In [85]:
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np

images_path = Path("./outputs")
classes = [str(c.stem) for c in images_path.iterdir()]
epochs = np.linspace(start=1, stop=200).astype(int)  # Generates 5 epochs from 1 to 10
my_classes = ['bottle', 'zipper', 'capsule', 'transistor']
classes

['tile',
 'screw',
 'cable',
 'grid',
 'transistor',
 'pill',
 'carpet',
 'toothbrush',
 'capsule',
 'wood',
 'zipper',
 'metal_nut',
 'leather',
 'bottle',
 'hazelnut']

In [86]:
images = {
    c: list(images_path.joinpath(c).iterdir())
    for c in classes
}
for c in classes:
    images[c] = [list(images_path.joinpath(f"{c}/{e}").iterdir())[0] for e in epochs]

In [87]:
import imageio.v2 as imageio
from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import io

def add_text_to_image(img, text, position=(0, 0), font_size=40):
    draw = ImageDraw.Draw(img)
    # Load a TrueType or OpenType font file, you can specify the path to a .ttf or .otf file
    font = ImageFont.truetype("fonts/arial.ttf", font_size)
    
    # Determine the size of the text to create a black rectangle background
    text_bbox = draw.textbbox((0, 0), text, font=font)
    text_width = text_bbox[2] - text_bbox[0]
    text_height = text_bbox[3] - text_bbox[1]
    rectangle_size = (text_width + 10, text_height + 10)  # Add some padding

    # Calculate position for the rectangle and text
    rectangle_position = (img.width - rectangle_size[0], 0)
    text_position = (rectangle_position[0] + 5, rectangle_position[1] + 5)

    # Draw the black rectangle
    draw.rectangle([rectangle_position, (rectangle_position[0] + rectangle_size[0], rectangle_position[1] + rectangle_size[1])], fill="black")
    
    # Draw the text
    draw.text(text_position, text, fill="white", font=font)
    
    return img

def create_gif(image_paths, output_path, epochs, duration=0.5, resolution=(128, 128)):
    images = []
    for image_path, epoch in zip(image_paths, epochs):
        img = Image.open(image_path)
        img = img.resize(resolution, Image.LANCZOS)
        
        # Add epoch number text to image
        img = add_text_to_image(img, f"Epoch: {epoch:4d}", font_size=40)
        
        # Convert PIL image to bytes and then read with imageio
        with io.BytesIO() as output:
            img.save(output, format="PNG")
            imageio_image = imageio.imread(output.getvalue(), format='png')
            images.append(imageio_image)

    # Save the images as a GIF
    imageio.mimsave(output_path, images, duration=duration, loop=0)

# Assuming 'classes' and 'images' are defined elsewhere in your code
for c in classes:
    image_paths = images[c]
    epochs = range(len(image_paths))  # Generate dummy epochs if not provided
    Path("gifs/").mkdir(exist_ok=True)
    # Pass the epoch number to the create_gif function
    create_gif(image_paths, f"gifs/{c}.gif", epochs, duration=1, resolution=(512, 512))
